In [2]:
import requests

In [3]:
SMART_BASE_URL = "https://smart.argo.run/v/r4/sim/eyJoIjoiMSIsImIiOiI2ODk4OTJiZC1kY2JlLTQxZmMtODY1MS0zOGExZDA4OTM4NTQiLCJqIjoiMSJ9/fhir/"

## Scenario 0

In [4]:
DISCOVERY_PATH = ".well-known/smart-configuration"

In [5]:
smart_config = requests.get(SMART_BASE_URL + DISCOVERY_PATH).json()

Payload includes `permission_v2` and `authorize-post` capabilities ✔️:

In [7]:
smart_config["capabilities"]

['launch-ehr',
 'launch-standalone',
 'client-public',
 'client-confidential-symmetric',
 'context-passthrough-banner',
 'context-passthrough-style',
 'context-ehr-patient',
 'context-ehr-encounter',
 'context-standalone-patient',
 'context-standalone-encounter',
 'permission-offline',
 'permission-patient',
 'permission-user',
 'permission-v2',
 'authorize-post']

`code_challenge_methods_supported` includes `S256` ✔️:

In [9]:
smart_config["code_challenge_methods_supported"]

['S256']

`introscpection_endpoint` included ✔️:

In [11]:
smart_config["introspection_endpoint"]

'https://smart.argo.run/v/r4/sim/eyJoIjoiMSIsImIiOiI2ODk4OTJiZC1kY2JlLTQxZmMtODY1MS0zOGExZDA4OTM4NTQiLCJqIjoiMSJ9/auth/introspect'